In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
output_dir = "./frames/"

cap = cv2.VideoCapture("./test.mp4")
if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

frame_count = 0
while True:
    frame_ready, frame = cap.read()  # Read the frame
    if not frame_ready:
        print("End of video or error reading frame.")
        break

    # Convert to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    #plt.imshow(gray_frame, cmap='gray')
    #plt.show()

    #print(gray_frame.shape)

    # Save frame as a numpy array
    frame_path = os.path.join(output_dir, f"frame_{frame_count:04d}.npy")
    np.save(frame_path, gray_frame)

    frame_count += 1

    # Show progress
    print(f"Processed frame {frame_count}")

    # Exit on key press (optional)
    if cv2.waitKey(10) == 27:  # Escape key
        break

cap.release()
print(f"Frames saved in directory: {output_dir}")


Processed frame 1
Processed frame 2
Processed frame 3
Processed frame 4
Processed frame 5
Processed frame 6
Processed frame 7
Processed frame 8
Processed frame 9
Processed frame 10
Processed frame 11
Processed frame 12
Processed frame 13
Processed frame 14
Processed frame 15
Processed frame 16
Processed frame 17
Processed frame 18
Processed frame 19
Processed frame 20
Processed frame 21
Processed frame 22
Processed frame 23
Processed frame 24
Processed frame 25
Processed frame 26
Processed frame 27
Processed frame 28
Processed frame 29
Processed frame 30
Processed frame 31
Processed frame 32
Processed frame 33
Processed frame 34
Processed frame 35
Processed frame 36
Processed frame 37
Processed frame 38
Processed frame 39
Processed frame 40
Processed frame 41
Processed frame 42
Processed frame 43
Processed frame 44
Processed frame 45
Processed frame 46
Processed frame 47
Processed frame 48
Processed frame 49
Processed frame 50
Processed frame 51
Processed frame 52
Processed frame 53
Pr

In [2]:
from PIL import Image, ImageDraw, ImageFont

def ascii_to_image(ascii_art, font_path='cour.ttf', font_size=12, bg_color='black', text_color='white'):
    """
    Converts a 2D list (or list of strings) of ASCII characters into an image.
    Adjust font_path to point to a monospaced font on your system (e.g., Courier).
    """
    # Ensure ascii_art is a list of strings (each row)
    if isinstance(ascii_art[0], list):
        ascii_art = ["".join(row) for row in ascii_art]
    
    # Determine the size of the image.
    # Here, we assume each character takes roughly (font_size, font_size) pixels.
    rows = len(ascii_art)
    cols = max(len(row) for row in ascii_art)
    image_width = cols * font_size
    image_height = rows * font_size
    
    image = Image.new("RGB", (image_width, image_height), color=bg_color)
    draw = ImageDraw.Draw(image)
    
    try:
        font = ImageFont.truetype(font_path, font_size)
    except IOError:
        print("Could not load font, using default.")
        font = ImageFont.load_default()
    
    y = 0
    for line in ascii_art:
        draw.text((0, y), line, fill=text_color, font=font)
        y += font_size
    return image


In [3]:
# Define or import the load_ascii_mappings function if not already defined:
def load_ascii_mappings(ascii_csv_path):
    import numpy as np
    import pandas as pd
    df = pd.read_csv(ascii_csv_path)
    ascii_mappings = {
        3: [np.fromstring(row.strip('[]'), sep=' ') for row in df['pca_3']],
        5: [np.fromstring(row.strip('[]'), sep=' ') for row in df['pca_5']],
        7: [np.fromstring(row.strip('[]'), sep=' ') for row in df['pca_7']],
    }
    ascii_chars = df['char'].to_numpy()
    return ascii_mappings, ascii_chars

# Set the path to your CSV file containing the ASCII mappings
ascii_csv_path = "./ascii_characters_roman.csv"

# Load the mappings and characters
ascii_mappings, ascii_chars = load_ascii_mappings(ascii_csv_path)


/tmp/ipykernel_542439/4219830921.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
def process_frame(image, ascii_mappings, ascii_chars, num_col_patches=80, pca_size=5):
    if image.ndim != 2:
        raise ValueError("Input image must be a grayscale image.")

    original_a, original_b = image.shape  # Original height and width

    # Aspect ratio adjustment
    multiplier = (original_b * num_col_patches) / original_a
    k = round(multiplier)
    resized_b = num_col_patches * k
    resized_a = (original_a / original_b) * resized_b
    resized_a = round(resized_a / k) * k

    resized_image = cv2.resize(image, (int(resized_b), int(resized_a)))

    # Optional visualization block (commented or wrapped in try/except)
    try:
        resized_resized_image = cv2.resize(resized_image, (int(resized_b // k), int(resized_a // k)))
        print("Visualization image shape:", resized_resized_image.shape)
        # plt.imshow(resized_resized_image, cmap='gray')
        # plt.show()
    except Exception as e:
        print("Visualization skipped:", e)

    # Extract patches using the helper function
    patches, rows, cols = extract_patches(resized_image, k)
    
    # Correctly call calculate_ascii with all required arguments
    ascii_result = calculate_ascii(patches, pca_size, ascii_mappings, ascii_chars)
    
    # Convert the ASCII result to a 2D list
    processed_image = np.array(ascii_result).reshape(rows, cols).tolist()
    print("Processed frame to ASCII art, shape:", (rows, cols))
    
    return processed_image


In [5]:
import numpy as np
from sklearn.decomposition import PCA
import sys

def extract_patches(image, patch_size):
    """
    Extract patches from the image.
    
    Args:
        image (np.ndarray): Grayscale image array.
        patch_size (int): The size (height and width) of each patch.
    
    Returns:
        patches (list): List of flattened patches.
        rows (int): Number of patch rows.
        cols (int): Number of patch columns.
    """
    a, b = image.shape
    if a % patch_size != 0 or b % patch_size != 0:
        raise ValueError("Image dimensions must be divisible by patch size.")
    
    patches = [image[i:i+patch_size, j:j+patch_size].flatten()
               for i in range(0, a, patch_size)
               for j in range(0, b, patch_size)]
    return patches, a // patch_size, b // patch_size

def calculate_ascii(patches, pca_size, ascii_mappings, ascii_chars):
    """
    Calculate ASCII values for patches using PCA.
    
    Args:
        patches (list): List of flattened image patches.
        pca_size (int): Number of PCA components.
        ascii_mappings (dict): Mapping of PCA components to ASCII vector values.
        ascii_chars (np.ndarray): Array of ASCII characters.
    
    Returns:
        ascii_result (list): List of ASCII characters corresponding to each patch.
    """
    pca = PCA(n_components=pca_size)
    patches_pca = pca.fit_transform(patches)
    ascii_result = []
    for patch in patches_pca:
        min_dist = sys.maxsize
        min_char = None
        # Use the ascii mapping for the given PCA size
        for i, ascii_vec in enumerate(ascii_mappings[pca_size]):
            dist = np.linalg.norm(patch - ascii_vec)
            if dist < min_dist:
                min_dist = dist
                min_char = chr(ascii_chars[i])
        ascii_result.append(min_char)
    return ascii_result


In [6]:
import cv2
import numpy as np
import os

# Example function definitions (make sure these are defined or imported properly)
# def load_ascii_mappings(...): ...
# def process_frame(...): ...
# def ascii_to_image(...): ...

# Load your ASCII mappings
ascii_csv_path = "./ascii_characters_roman.csv"
ascii_mappings, ascii_chars = load_ascii_mappings(ascii_csv_path)

# Define the directory where frames are stored and output directory for ASCII images
input_dir = "./frames"
ascii_output_dir = "./ascii_frames"
os.makedirs(ascii_output_dir, exist_ok=True)

# For demonstration, try loading a single frame file (adjust the filename as needed)
frame_file = "frame_0000.npy"
frame_path = os.path.join(input_dir, frame_file)

try:
    frame = np.load(frame_path)
except Exception as e:
    print(f"Error loading {frame_path}: {e}")
    frame = None

if frame is None:
    print(f"Warning: Frame loaded from {frame_path} is None. Please check your input files.")
else:
    # Process the frame into ASCII art
    ascii_art = process_frame(frame, ascii_mappings, ascii_chars, num_col_patches=80, pca_size=5)
    # Convert ASCII art to an image (make sure to have a valid monospaced font file, e.g., 'cour.ttf')
    image = ascii_to_image(ascii_art, font_path='cour.ttf', font_size=12)
    # Save the image
    image_save_path = os.path.join(ascii_output_dir, f"{os.path.splitext(frame_file)[0]}.png")
    image.save(image_save_path)
    print(f"Saved ASCII art image to {image_save_path}")


Visualization image shape: (45, 80)


/home/sushovan/.local/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:640: RuntimeWarning: divide by zero encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var
/home/sushovan/.local/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:640: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var


Processed frame to ASCII art, shape: (45, 80)
Could not load font, using default.
Saved ASCII art image to ./ascii_frames/frame_0000.png


In [9]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import pandas as pd
from sklearn.decomposition import PCA
import sys

##############################
# Helper Functions and Setup #
##############################

def load_ascii_mappings(ascii_csv_path):
    """
    Load ASCII mappings and character list from a CSV file.
    The CSV is assumed to have columns: 'pca_3', 'pca_5', 'pca_7', and 'char'.
    Each row for pca_n should look like [0.1 0.2 0.3 ...].
    """
    df = pd.read_csv(ascii_csv_path)

    # Convert string representations like "[0.1 0.2 0.3]" into numpy arrays.
    ascii_mappings = {
        3: [np.fromstring(row.strip('[]'), sep=' ') for row in df['pca_3']],
        5: [np.fromstring(row.strip('[]'), sep=' ') for row in df['pca_5']],
        7: [np.fromstring(row.strip('[]'), sep=' ') for row in df['pca_7']],
    }
    # 'char' is expected to contain the numeric ASCII code or the actual character.
    # If it's a numeric ASCII code, use `chr(ascii_chars[i])` when mapping.
    ascii_chars = df['char'].to_numpy()
    return ascii_mappings, ascii_chars

def extract_patches(image, patch_size):
    """
    Divide the image into non-overlapping patches of size patch_size x patch_size.
    Returns a list of flattened patches and the number of patch rows and columns.
    """
    a, b = image.shape
    if a % patch_size != 0 or b % patch_size != 0:
        raise ValueError("Image dimensions must be divisible by patch size.")
    patches = [image[i:i+patch_size, j:j+patch_size].flatten()
               for i in range(0, a, patch_size)
               for j in range(0, b, patch_size)]
    return patches, a // patch_size, b // patch_size

def calculate_ascii(patches, pca_size, ascii_mappings, ascii_chars):
    """
    For each flattened patch, use PCA to reduce its dimension and map it to an ASCII character
    by finding the closest precomputed ASCII vector.
    """
    # Perform PCA on the patches
    pca = PCA(n_components=pca_size)
    patches_pca = pca.fit_transform(patches)

    ascii_result = []
    for patch in patches_pca:
        min_dist = float('inf')
        min_char = None

        # Compare the patch's PCA vector to each known ASCII PCA vector
        for i, ascii_vec in enumerate(ascii_mappings[pca_size]):
            dist = np.linalg.norm(patch - ascii_vec)
            if dist < min_dist:
                min_dist = dist
                # If 'char' column is numeric (ASCII code), convert to character with chr().
                # If 'char' column is already a character, use it directly: ascii_chars[i].
                try:
                    min_char = chr(int(ascii_chars[i]))
                except ValueError:
                    min_char = str(ascii_chars[i])  # if it's already a string/char
        ascii_result.append(min_char)
    return ascii_result

def process_frame(image, ascii_mappings, ascii_chars, num_col_patches=80, pca_size=5):
    """
    Process a single grayscale frame into a 2D ASCII art representation.
    The image is first resized based on the desired number of column patches.
    """
    if image.ndim != 2:
        raise ValueError("Input image must be a grayscale image.")

    original_a, original_b = image.shape

    # Determine a patch size (k) based on the desired number of column patches.
    # multiplier is how many pixels we want per patch in the "width" dimension.
    # This is a heuristic: (original_b * num_col_patches) / original_a
    multiplier = (original_b * num_col_patches) / original_a
    k = round(multiplier)
    
    # The new width = number of columns * patch_size
    resized_b = num_col_patches * k
    # The new height is scaled proportionally
    resized_a_float = (original_a / original_b) * resized_b
    # Make sure resized_a is divisible by k
    resized_a = round(resized_a_float / k) * k

    # Resize the image for processing (OpenCV expects integer dimensions)
    resized_image = cv2.resize(image, (int(resized_b), int(resized_a)))

    # Optional visualization (for debugging purposes)
    try:
        # Create a smaller version for quick preview (1 patch = 1 pixel).
        vis_img = cv2.resize(
            resized_image, 
            (int(resized_b // k), int(resized_a // k)),
            interpolation=cv2.INTER_NEAREST
        )
        print("Visualization image shape (rows x cols):", vis_img.shape)
        # Uncomment the next two lines to see the visualization:
        # plt.imshow(vis_img, cmap='gray')
        # plt.show()
    except Exception as e:
        print("Visualization skipped:", e)

    # Divide the resized image into patches and compute ASCII mapping.
    patches, rows, cols = extract_patches(resized_image, k)
    ascii_result = calculate_ascii(patches, pca_size, ascii_mappings, ascii_chars)
    
    # Convert the linear list into a 2D list (rows x cols)
    processed_image = np.array(ascii_result).reshape(rows, cols).tolist()
    print("Processed frame to ASCII art, shape (rows, cols):", (rows, cols))
    return processed_image

def ascii_to_image_centered(
    ascii_art,
    output_dir,
    frame_name,
    font_path='cour.ttf',
    font_size=12,
    bg_color='black',
    text_color='white',
    char_spacing_str="  "):
    """
    Convert a 2D ASCII art (list of lists or list of strings) into a properly centered image
    and also save the ASCII text file and the image in output_dir using frame_name as the base.
    """

    # If ascii_art is a 2D list, join each row into a string
    if isinstance(ascii_art[0], list):
        ascii_art = ["".join(row) for row in ascii_art]
    
    # Optionally insert spacing between characters (e.g., "  ")
    spaced_ascii_art = []
    for line in ascii_art:
        spaced_line = char_spacing_str.join(list(line))
        spaced_ascii_art.append(spaced_line)
    ascii_art = spaced_ascii_art
    
    # Calculate text dimensions
    rows = len(ascii_art)
    cols = max(len(line) for line in ascii_art)
    
    # This is a heuristic for approximate width/height.
    # Each character is ~ (font_size/2) wide in many monospaced fonts.
    image_width = cols * (font_size // 2)  
    image_height = rows * font_size
    
    # Add padding so the text isn't right on the border
    padding_x = font_size * 2
    padding_y = font_size * 2
    
    # Create a new image for the ASCII art
    image = Image.new(
        mode="RGB",
        size=(image_width + 2 * padding_x, image_height + 2 * padding_y),
        color=bg_color
    )
    draw = ImageDraw.Draw(image)
    
    # Attempt to load custom font. If it fails, use default.
    try:
        font = ImageFont.truetype(font_path, font_size)
    except IOError:
        print("Could not load font at '{}'. Using default font.".format(font_path))
        font = ImageFont.load_default()
    
    # Draw each line, centered horizontally
    y_cursor = padding_y
    for line in ascii_art:
        text_width = draw.textlength(line, font=font)
        # Center horizontally
        x_cursor = (image_width - text_width) // 2 + padding_x
        draw.text((x_cursor, y_cursor), line, fill=text_color, font=font)
        y_cursor += font_size
    
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # Save ASCII text to file
    ascii_text_path = os.path.join(output_dir, f"{frame_name}.txt")
    with open(ascii_text_path, "w", encoding="utf-8") as f:
        f.write("\n".join(ascii_art))
    print(f"ASCII text saved to {ascii_text_path}")
    
    # Save ASCII image
    ascii_image_path = os.path.join(output_dir, f"{frame_name}.png")
    image.save(ascii_image_path)
    print(f"ASCII image saved to {ascii_image_path}")
    
    return image

###########################################
# Main Loop: Process All 20-Second Frames #
###########################################

def main():
    # Adjust paths as needed
    input_dir = "./frames"                # Folder containing .npy frames
    ascii_output_dir = "./ascii_frames"   # Where ASCII images and text files will go
    ascii_csv_path = "./ascii_characters_roman.csv"  # CSV with PCA vectors and chars

    # Create the output directory if needed
    os.makedirs(ascii_output_dir, exist_ok=True)

    # Load the ASCII mappings
    ascii_mappings, ascii_chars = load_ascii_mappings(ascii_csv_path)

    # Get a sorted list of all .npy frame files
    frame_files = sorted([f for f in os.listdir(input_dir) if f.endswith('.npy')])

    for frame_file in frame_files:
        frame_path = os.path.join(input_dir, frame_file)
        
        # Load the frame (assumed to be a grayscale image saved as a numpy array)
        try:
            frame = np.load(frame_path)
        except Exception as e:
            print(f"Error loading {frame_path}: {e}")
            continue
        
        if frame is None:
            print(f"Warning: Frame loaded from {frame_path} is None. Skipping.")
            continue
        
        # Derive a name for saving results (e.g., "frame_0001")
        frame_name = os.path.splitext(frame_file)[0]
        
        # Process the frame into ASCII art (you can tweak num_col_patches, pca_size)
        ascii_art = process_frame(frame, ascii_mappings, ascii_chars, 
                                  num_col_patches=80, pca_size=5)
        
        # Convert the ASCII art to an image and also save the .txt and .png
        image = ascii_to_image_centered(
            ascii_art, 
            output_dir=ascii_output_dir, 
            frame_name=frame_name,
            font_path='cour.ttf',         # Change to a valid TTF on your system
            font_size=12, 
            bg_color='black', 
            text_color='white',
            char_spacing_str="  "
        )
        
        print(f"Finished processing: {frame_file}")

if __name__ == "__main__":
    main()


Visualization image shape (rows x cols): (45, 80)


/home/sushovan/.local/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:640: RuntimeWarning: divide by zero encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var
/home/sushovan/.local/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:640: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var


Processed frame to ASCII art, shape (rows, cols): (45, 80)
Could not load font at 'cour.ttf'. Using default font.
ASCII text saved to ./ascii_frames/frame_0000.txt
ASCII image saved to ./ascii_frames/frame_0000.png
Finished processing: frame_0000.npy
Visualization image shape (rows x cols): (45, 80)
Processed frame to ASCII art, shape (rows, cols): (45, 80)
Could not load font at 'cour.ttf'. Using default font.
ASCII text saved to ./ascii_frames/frame_0001.txt
ASCII image saved to ./ascii_frames/frame_0001.png
Finished processing: frame_0001.npy
Visualization image shape (rows x cols): (45, 80)
Processed frame to ASCII art, shape (rows, cols): (45, 80)
Could not load font at 'cour.ttf'. Using default font.
ASCII text saved to ./ascii_frames/frame_0002.txt
ASCII image saved to ./ascii_frames/frame_0002.png
Finished processing: frame_0002.npy
Visualization image shape (rows x cols): (45, 80)
Processed frame to ASCII art, shape (rows, cols): (45, 80)
Could not load font at 'cour.ttf'. Usi

In [1]:
import os
import cv2
import ffmpeg
import numpy as np

def extract_audio_ffmpeg(input_video_path, audio_output_path):
    """
    Extracts audio from a video file using FFmpeg.
    """
    try:
        (
            ffmpeg
            .input(input_video_path)
            .output(audio_output_path, format="mp3", acodec="mp3")
            .run(overwrite_output=True, quiet=True)
        )
        print(f"✅ Extracted audio saved to: {audio_output_path}")
    except ffmpeg.Error as e:
        print(f"❌ FFmpeg Error: {e}")

def create_video_from_frames_opencv(frames_dir, output_video_path, fps=23.5):
    """
    Creates a video from PNG frames using OpenCV.
    """
    frame_files = sorted([
        f for f in os.listdir(frames_dir) if f.lower().endswith('.png')
    ])

    if not frame_files:
        raise ValueError(f"❌ No PNG frames found in {frames_dir}")

    first_frame = cv2.imread(os.path.join(frames_dir, frame_files[0]))
    height, width, _ = first_frame.shape

    # Define the video writer
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    for frame_file in frame_files:
        frame_path = os.path.join(frames_dir, frame_file)
        frame = cv2.imread(frame_path)
        if frame is None:
            print(f"⚠️ Warning: Could not read {frame_path}, skipping.")
            continue
        video_writer.write(frame)

    video_writer.release()
    print(f"✅ Video saved to: {output_video_path}")

def merge_video_audio_ffmpeg(video_path, audio_path, output_final_video):
    """
    Merges video and audio using FFmpeg.
    """
    try:
        # Properly define video and audio inputs separately
        video_input = ffmpeg.input(video_path)
        audio_input = ffmpeg.input(audio_path)

        # Merge them into a single output
        (
            ffmpeg
            .output(video_input, audio_input, output_final_video, vcodec="libx264", acodec="aac", strict="experimental")
            .run(overwrite_output=True, quiet=True)
        )
        print(f"✅ Final video with audio saved to: {output_final_video}")
    except ffmpeg.Error as e:
        print(f"❌ FFmpeg Error: {e}")

if __name__ == "__main__":
    # Paths (adjust as needed)
    original_video_path = "test.mp4"
    extracted_audio_path = "extracted_audio.mp3"
    ascii_frames_dir = "./ascii_frames"
    silent_video_path = "ascii_video_silent.mp4"
    output_video_path = "ascii_with_audio.mp4"
    fps = 23.4  # Adjust frame rate as needed

    # Step 1: Extract audio from the original video
    extract_audio_ffmpeg(original_video_path, extracted_audio_path)

    # Step 2: Create a video from the ASCII frames (silent video)
    create_video_from_frames_opencv(ascii_frames_dir, silent_video_path, fps)

    # Step 3: Merge silent video and extracted audio
    merge_video_audio_ffmpeg(silent_video_path, extracted_audio_path, output_video_path)

    print(f"🎉 Finished! The final video is saved to: {output_video_path}")


✅ Extracted audio saved to: extracted_audio.mp3
✅ Video saved to: ascii_video_silent.mp4
✅ Final video with audio saved to: ascii_with_audio.mp4
🎉 Finished! The final video is saved to: ascii_with_audio.mp4
